In [10]:
import numpy as np
from numpy import pi
from numpy.linalg import norm
from matplotlib import pyplot as plt
from scipy import signal
from scipy.ndimage import gaussian_filter

import simulation
from algorithm import *

## 仿真实验


### 声场仿真

设置参数


In [11]:
f = 37500  # 声源频率
T = 1  # 采样时长
r = 300  # 距离
theta = np.deg2rad(60)  # 角度

d = 0.5
K = 0.6 / d
c = 1500  # 声速
fs = 500e3  # 采样频率500k

#### 几何模型

- 阵元1, 2, 3坐标分别为(d1, 0), (d2, 0), (d3, 0), 其中阵元1, 2在x轴负半轴.
- theta为声源S的方位角 (极坐标下角度)

In [12]:
d1, d2, d3 = -(K + 1) * d / 2, -(K - 1) * d / 2, (K + 1) * d / 2
S = np.array([r * np.cos(theta), r * np.sin(theta)])

r1 = float(norm(S - [d1, 0]))
r2 = float(norm(S - [d2, 0]))
r3 = float(norm(S - [d3, 0]))

# 真实时延
t12 = (r1 - r2) / c
t23 = (r2 - r3) / c

#### 生成各阵元电信号AD采样后信号

In [13]:
# 三阵元处采样信号序列 # TODO: 改成用同一个函数生成, 包含阵元的坐标, 生成信号矩阵
weights = [2, 2]
x1 = simulation.sig_gen(c, f, r1, fs, T, weights, 0)
x2 = simulation.sig_gen(c, f, r2, fs, T, weights, 1000)
x3 = simulation.sig_gen(c, f, r3, fs, T, weights, 1000000)

##### 信号图像

In [14]:
# plt.plot(x1)
# plt.plot(x2)
# plt.plot(x3)
# plt.show()

### 定位

#### 时延估计

In [15]:
period, T_on = 1, 10e-3

tau12_hat = chirp_delay_estimation(x1, x2, fs, f, period, T_on, 3)
tau23_hat = chirp_delay_estimation(x2, x3, fs, f, period, T_on, 3)

In [16]:
phi12_hat, phi23_hat = tau12_hat * f, tau23_hat * f

In [17]:
print(f'真实时延差tau12: \t{t12}\n相关时延差tau12\': \t{tau12_hat}\n差异: {np.abs(t12 - tau12_hat) / t12 * 100:.2f}%')
print(f'真实时延差tau23: \t{t23}\n相关时延差tau23\': \t{tau23_hat}\n差异: {np.abs(t23 - tau23_hat) / t23 * 100:.2f}%')

真实时延差tau12: 	0.00016691643581710498
相关时延差tau12': 	0.0001673333333333249
差异: 0.25%
真实时延差tau23: 	0.00019974976869730198
相关时延差tau23': 	0.00020000000000000573
差异: 0.13%


#### 解算方位

In [30]:
r_far, theta_far = far_locate(tau12_hat, tau23_hat, c, K, d)
r_near, theta_near = near_locate(tau12_hat, tau23_hat, c, K, d)
print(f'----真实----\nr: {r:.3f}, theta: {np.rad2deg(theta):.3f}')
print(f'----远场----\nr: {r_far:.3f}, theta: {np.rad2deg(theta_far):.3f}')
print(f'----近场----\nr: {r_near:.3f}, theta: {np.rad2deg(theta_near):.3f}\n方位估计绝对误差: \t{np.rad2deg(theta - theta_near):.3f}度\n方位估计相对误差: \t{(theta - theta_near)/theta * 100:.3f}%')

----真实----
r: 300.000, theta: 60.000
----远场----
r: 206.000, theta: 59.940
----近场----
r: 206.025, theta: 59.916
方位估计绝对误差: 	0.084度
方位估计相对误差: 	0.141%
